# Ollama server

In [6]:
%pip -q install python-magic tabulate pdf2image pytesseract llama-index llama-index-readers-web llama-index-llms-ollama llama-index-embeddings-huggingface llama-index-readers-file unstructured einops datasets huggingface_hub sentence-transformers

In [2]:
! curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0>>> Downloading ollama...
100 10941    0 10941    0     0  33039      0 --:--:-- --:--:-- --:--:-- 33054
############################################################################################# 100.0%
>>> Installing ollama to /usr/local/bin...
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:
import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])


In [4]:
ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [5]:
!ollama pull llama3

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a...   0% ▕

In [6]:
!ollama list

NAME         	ID          	SIZE  	MODIFIED           
llama3:latest	365c0bd3c000	4.7 GB	About a minute ago	


# Libraries

In [3]:
from llama_index.core import SimpleDirectoryReader
from llama_index.core import VectorStoreIndex
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import Settings
from llama_index.readers.web import SimpleWebPageReader
from llama_index.readers.file import (
    DocxReader,
    HWPReader,
    PDFReader,
    EpubReader,
    FlatReader,
    HTMLTagReader,
    ImageCaptionReader,
    ImageReader,
    ImageVisionLLMReader,
    IPYNBReader,
    MarkdownReader,
    MboxReader,
    PptxReader,
    PandasCSVReader,
    VideoAudioReader,
    UnstructuredReader,
    PyMuPDFReader,
    ImageTabularChartReader,
    XMLReader,
    PagedCSVReader,
    CSVReader,
    RTFReader,
)
from llama_index.core import (
    load_index_from_storage,
    StorageContext,
)


# Trying Embedding models from MTEB

In [9]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [3]:
!huggingface-cli whoami

ballav


# If want to download the model locally

In [2]:
# !huggingface-cli download meta-llama/Meta-Llama-3-8B --include "original/*" --local-dir Meta-Llama-3-8B
!huggingface-cli download meta-llama/Meta-Llama-3-8B --local-dir Meta-Llama-3-8B


Fetching 17 files:   0% 0/17 [00:00<?, ?it/s]Downloading 'README.md' to 'Meta-Llama-3-8B/.huggingface/download/README.md.53e7e6ec18e5d060bc5690314c0e04f2df3326bf.incomplete'

README.md: 100% 36.5k/36.5k [00:00<00:00, 5.15MB/s]
Download complete. Moving file to Meta-Llama-3-8B/README.md

USE_POLICY.md: 100% 4.70k/4.70k [00:00<00:00, 18.1MB/s]
Download complete. Moving file to Meta-Llama-3-8B/USE_POLICY.md

LICENSE: 100% 7.80k/7.80k [00:00<00:00, 34.2MB/s]
Download complete. Moving file to Meta-Llama-3-8B/LICENSE

config.json: 100% 654/654 [00:00<00:00, 5.30MB/s]
Download complete. Moving file to Meta-Llama-3-8B/config.json

model-00002-of-00004.safetensors:   0% 0.00/5.00G [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0% 0.00/4.98G [00:00<?, ?B/s]


generation_config.json: 100% 177/177 [00:00<00:00, 1.23MB/s]
Download complete. Moving file to Meta-Llama-3-8B/generation_config.json



model-00003-of-00004.safetensors:   0% 0.00/4.92G [00:00<?, ?B/s]



model.safetensors.index.json

In [ ]:
Settings.embed_model = HuggingFaceEmbedding(model_name="nvidia/NV-Embed-v1", trust_remote_code=True, use_auth_token=True)

/usr/local/lib/python3.10/dist-packages/sentence_transformers/SentenceTransformer.py:99: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v3 of SentenceTransformers.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [ ]:
from transformers import LlamaForCausalLM, LlamaTokenizer

model = LlamaForCausalLM.from_pretrained("/content/Meta-Llama-3-8B(1)")
tokenizer = LlamaTokenizer.from_pretrained("/content/Meta-Llama-3-8B(1)")


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

# Data loading and storing indexes

# 1. Tablular data from webpage




*   Detecting the tables
*   Saving it as csv
*   Mapping the csv files with their corresponding url
*   Indexing of these csv data



In [ ]:
file_to_url_mapping={}

In [ ]:
def table_to_csv_converter(url,file_to_url_mapping):

  from bs4 import BeautifulSoup
  import requests
  import pandas as pd
  import os

  url = url
  page = requests.get(url)
  soup = BeautifulSoup(page.text, 'html')

  # Detect all tables on the webpage
  all_tables = soup.find_all('table')
  print(f"Total number of tables: {len(all_tables)}")

  # Create a directory to save the CSV files
  directory = 'tabular_data_csv'
  if not os.path.exists(directory):
      os.makedirs(directory)

  # Iterate over each table
  for i, table in enumerate(all_tables):
      # Get the caption of the table
      table_caption = table.find('caption').text.strip()

      world_titles = table.find_all('th')
      world_table_titles = [title.text.strip() for title in world_titles]

      df = pd.DataFrame(columns=world_table_titles)

      column_data = table.find_all('tr')
      for row in column_data[1:]:
          row_data = row.find_all('td')
          individual_row_data = [data.text.strip() for data in row_data]
          df.loc[len(df)] = individual_row_data

      # Save the DataFrame as a CSV file with the table caption as the file name
      csv_file = os.path.join(directory, f'{table_caption}.csv')
      df.to_csv(csv_file, index=False)
      print(f"CSV file saved: {csv_file}")
      #file_to_url_mapping.append({ f'{table_caption}.csv':url})
      file_to_url_mapping[f'{table_caption}.csv'] = url

In [ ]:
url_with_table = ['https://maharashtra.nic.in/',
 'https://maharashtra.nic.in/services/',
 'https://maharashtra.nic.in/profile/',
 'https://www.nic.in/servicecontents/nicnet/',
 'https://www.nic.in/servicecontents/data-centre/',
 'https://www.nic.in/servicecontents/national-cloud/',
 'https://www.nic.in/servicecontents/messaging/',
 'https://www.nic.in/servicecontents/remote-sensing-gis/',
 'https://www.nic.in/servicecontents/webcast/',
 'https://www.nic.in/servicecontents/nkn/',
 'https://www.nic.in/servicecontents/command-and-control-centre/',
 'https://www.nic.in/servicecontents/government-local-area-networks-lans/',
 'https://www.nic.in/servicecontents/video-conferencing/',
 'https://www.nic.in/servicecontents/security/',
 'https://www.nic.in/servicecontents/centralised-aadhaar-vault/',
 'https://maharashtra.nic.in/infrastructure/',
 'https://maharashtra.nic.in/news-update/',
 'https://maharashtra.nic.in/events/',
 'https://maharashtra.nic.in/awards/',
 'https://www.nic.in/servicecontents/domain-registration/',
 'https://maharashtra.nic.in/contact-us/',
 'https://maharashtra.nic.in/website-policies/',
 'https://maharashtra.nic.in/help/',
 'https://maharashtra.nic.in/web-information-manager/',
 'https://maharashtra.nic.in/directory/',
 'https://maharashtra.nic.in/rti/',
 'https://maharashtra.nic.in/district-centres/',
 'https://maharashtra.nic.in/photo-gallery/',
 'https://maharashtra.nic.in/video-gallery/', 'https://maharashtra.nic.in/organization-structure/']

In [ ]:
for url in url_with_table:
  table_to_csv_converter(url,file_to_url_mapping)

Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 0
Total number of tables: 2
CSV file saved: tabular_data_csv/Screen Reader Information.csv
CSV file saved: tabular_data_csv/Plug-in for alternate document types.csv
Total number of tables: 0
Total number of tables: 37
CSV file saved: tabular_data_csv/SIO.csv
CSV file saved: tabular_data_csv/Maharashtra State Centre, Mumbai.csv
CSV file saved: tabular_data_csv/Ahmadnagar.csv
CSV file saved: tabular_data_csv/Akola.csv
CSV file s

In [ ]:
file_to_url_mapping

{'Screen Reader Information.csv': 'https://maharashtra.nic.in/help/',
 'Plug-in for alternate document types.csv': 'https://maharashtra.nic.in/help/',
 'SIO.csv': 'https://maharashtra.nic.in/directory/',
 'Maharashtra State Centre, Mumbai.csv': 'https://maharashtra.nic.in/directory/',
 'Ahmadnagar.csv': 'https://maharashtra.nic.in/directory/',
 'Akola.csv': 'https://maharashtra.nic.in/directory/',
 'Amravati.csv': 'https://maharashtra.nic.in/directory/',
 'Aurangabad.csv': 'https://maharashtra.nic.in/directory/',
 'Beed.csv': 'https://maharashtra.nic.in/directory/',
 'Bhandara.csv': 'https://maharashtra.nic.in/directory/',
 'Buldana.csv': 'https://maharashtra.nic.in/directory/',
 'Chandrapur.csv': 'https://maharashtra.nic.in/directory/',
 'Dhule.csv': 'https://maharashtra.nic.in/directory/',
 'Gadchiroli.csv': 'https://maharashtra.nic.in/directory/',
 'Gondia.csv': 'https://maharashtra.nic.in/directory/',
 'Hingoli.csv': 'https://maharashtra.nic.in/directory/',
 'Jalgaon.csv': 'https:/

Indexing of the CSV file

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader
import os
from llama_index.core import (
    load_index_from_storage,
    StorageContext,
)

Settings.llm = Ollama(model="llama3", temperature=0, request_timeout=500.0)
Settings.embed_model = HuggingFaceEmbedding(model_name="nvidia/NV-Embed-v1", trust_remote_code=True, use_auth_token=True)

# Paged CSV Reader example
parser = PagedCSVReader()

file_extractor = {".csv": parser}  # Add other CSV formats as needed
documents = SimpleDirectoryReader(
    "./tabular_data_csv", file_extractor=file_extractor
).load_data()
documents

# Create and save the VectorStoreIndex for the loaded data
directory_index = VectorStoreIndex.from_documents(documents, llm=Settings.llm, embed_model=Settings.embed_model)
#  save index1 to disk
directory_index.set_index_id("vector_index")
directory_index.storage_context.persist("./tabular_data_csv_index")






/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

# 2. Text data from the webpages

In [ ]:
from llama_index.core import VectorStoreIndex
from llama_index.readers.web import SimpleWebPageReader
import os
from llama_index.core import (
    SimpleDirectoryReader,
    load_index_from_storage,
    StorageContext,
)
from llama_index.core import download_loader

Settings.llm = Ollama(model="llama3", temperature=0, request_timeout=500.0)
Settings.embed_model = HuggingFaceEmbedding(model_name="nvidia/NV-Embed-v1", trust_remote_code=True, use_auth_token=True)

# URLs and corresponding link names
urls = {
    "https://maharashtra.nic.in/": "Home_index",
    "https://maharashtra.nic.in/services/": "Services_index",
    "https://maharashtra.nic.in/profile/": "profile_index",
    "https://www.nic.in/servicecontents/nicnet/": "nicnet_index",
    "https://www.nic.in/servicecontents/data-centre/": "data-centre_index",
    "https://www.nic.in/servicecontents/national-cloud/": "national-cloud_index",
    "https://www.nic.in/servicecontents/messaging/": "messaging_index",
    "https://www.nic.in/servicecontents/remote-sensing-gis/": "remote-sensing-gis_index",
    "https://www.nic.in/servicecontents/webcast/": "webcast_index",
    "https://www.nic.in/servicecontents/nkn/": "nkn_index",
    "https://www.nic.in/servicecontents/command-and-control-centre/": "command-and-control-centre_index",
    "https://www.nic.in/servicecontents/government-local-area-networks-lans/": "government-local-area-networks-lans_index",
    "https://www.nic.in/servicecontents/video-conferencing/": "video-conferencing_index",
    "https://www.nic.in/servicecontents/security/": "security_index",
    "https://www.nic.in/servicecontents/centralised-aadhaar-vault/": "centralised-aadhaar-vault_index",
    "https://maharashtra.nic.in/infrastructure/": "infrastructure_index",
    "https://maharashtra.nic.in/news-update/": "news-update_index",
    "https://maharashtra.nic.in/events/": "events_index",
    "https://maharashtra.nic.in/awards/": "awards_index",
    "https://www.nic.in/servicecontents/domain-registration/": "domain_registration_index",
    "https://maharashtra.nic.in/contact-us/": "contact-us_index",
    "https://maharashtra.nic.in/website-policies/": "website-policies_index",
    "https://maharashtra.nic.in/help/": "help_index",
    "https://maharashtra.nic.in/web-information-manager/": "web-information-manager_index",
    "https://maharashtra.nic.in/directory/": "directory_index",
    "https://maharashtra.nic.in/rti/": "rti_index",
    "https://maharashtra.nic.in/district-centres/": "district-centres_index",
    "https://maharashtra.nic.in/photo-gallery/": "photo-gallery_index",
    "https://maharashtra.nic.in/video-gallery/": "video-gallery_index",
    "https://maharashtra.nic.in/organization-structure/": "organization-structure_index"
}


#  Load data from multiple URLs
documents = []
BeautifulSoupWebReader = download_loader("BeautifulSoupWebReader")
loader = BeautifulSoupWebReader()
for url in urls:
    documents.extend(loader.load_data(urls=[url]))



# Define the directory path to save the documents and indexes
save_directory = "/content/docs"
index_directory = "/content/index"
if not os.path.exists(save_directory):
    os.makedirs(save_directory)

# Load data from the specified URLs and save each document with the link name
for url, link_name in urls.items():
    documents = loader.load_data(urls=[url])
    file_path = os.path.join(save_directory, f"{link_name}.txt")
    with open(file_path, "w", encoding="utf-8") as file:
        for doc in documents:
            file.write(doc.text + "\n")

    # Create and save the VectorStoreIndex for the loaded data
    index = VectorStoreIndex.from_documents(documents, llm=Settings.llm, embed_model=Settings.embed_model)

    # Create a unique directory for each index
    index_folder = os.path.join(index_directory, link_name)
    if not os.path.exists(index_folder):
        os.makedirs(index_folder)

    # Save the index to disk
    index.set_index_id("vector_index")
    index.storage_context.persist(index_folder)

    print(f"Data from {url} saved successfully to: {file_path}")
    print(f"Index saved successfully to: {index_directory}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
<ipython-input-14-9dbbe3ea6c5d>:51: DeprecationWarning: Call to deprecated function (or staticmethod) download_loader. (`download_loader()` is deprecated. Please install tool using pip install directly instead.)
  BeautifulSoupWebReader = download_loader("BeautifulSoupWebReader")


Data from https://maharashtra.nic.in/ saved successfully to: /content/docs/Home_index.txt
Index saved successfully to: /content/index
Data from https://maharashtra.nic.in/services/ saved successfully to: /content/docs/Services_index.txt
Index saved successfully to: /content/index
Data from https://maharashtra.nic.in/profile/ saved successfully to: /content/docs/profile_index.txt
Index saved successfully to: /content/index
Data from https://www.nic.in/servicecontents/nicnet/ saved successfully to: /content/docs/nicnet_index.txt
Index saved successfully to: /content/index
Data from https://www.nic.in/servicecontents/data-centre/ saved successfully to: /content/docs/data-centre_index.txt
Index saved successfully to: /content/index
Data from https://www.nic.in/servicecontents/national-cloud/ saved successfully to: /content/docs/national-cloud_index.txt
Index saved successfully to: /content/index
Data from https://www.nic.in/servicecontents/messaging/ saved successfully to: /content/docs/me

# Creating a zip of colab folder

In [ ]:
import shutil
import os
import zipfile

# Specify the parent folder path
parent_folder = '/content/Meta-Llama-3-8B(1)'

# Create a zip file
zip_file = zipfile.ZipFile('Meta-Llama-3-8B.zip', 'w')

# Iterate over the parent folder and its subfolders
for root, dirs, files in os.walk(parent_folder):
    for file in files:
        file_path = os.path.join(root, file)
        zip_file.write(file_path, os.path.relpath(file_path, parent_folder))

zip_file.close()


In [ ]:
from google.colab import files
files.download("Meta-Llama-3-8B.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# 3. Unstructured Webpage

In [ ]:
from llama_index.readers.web import UnstructuredURLLoader

# Initialize settings for LlamaIndex
Settings.llm = Ollama(model="llama3", temperature=0, request_timeout=500.0)
Settings.embed_model = HuggingFaceEmbedding(model_name="nvidia/NV-Embed-v1", trust_remote_code=True, use_auth_token=True)

urls = [
    "https://maharashtra.nic.in/organization-structure/",
]

loader = UnstructuredURLLoader(
    urls=urls, continue_on_failure=False, headers={"User-Agent": "value"}
)
documents = loader.load_data()
# Create a VectorStoreIndex from the documents
index = VectorStoreIndex.from_documents(documents, llm=Settings.llm, embed_model=Settings.embed_model)

index.set_index_id("vector_index")
index.storage_context.persist("./organization-structure_index")

# Extracting folder names

In [ ]:
import os

# Specify the path to the parent directory
parent_dir = '/content/index'

# Initialize an empty list to store the folder names
folder_names = []

# Iterate over the contents of the parent directory
for item in os.listdir(parent_dir):
    # Check if the item is a directory
    if os.path.isdir(os.path.join(parent_dir, item)):
        # Add the directory name to the list
        folder_names.append(item)

# Print the list of folder names
print(folder_names)
print(len(folder_names))


['organization-structure_index', 'remote-sensing-gis_index', 'awards_index', 'messaging_index', 'nicnet_index', 'events_index', 'help_index', 'domain_registration_index', 'website-policies_index', 'command-and-control-centre_index', 'centralised-aadhaar-vault_index', 'national-cloud_index', 'contact-us_index', 'photo-gallery_index', 'data-centre_index', 'security_index', 'webcast_index', 'infrastructure_index', 'video-conferencing_index', 'Services_index', 'district-centres_index', 'Home_index', 'video-gallery_index', 'directory_index', 'profile_index', 'web-information-manager_index', 'nkn_index', 'rti_index', 'news-update_index', 'government-local-area-networks-lans_index']
30


# extracting folder paths

In [ ]:
import os

# Specify the path to the parent directory
parent_dir = '/content/index'

# Initialize an empty list to store the folder paths
folder_paths = []

# Iterate over the contents of the parent directory
for item in os.listdir(parent_dir):
    # Check if the item is a directory
    if os.path.isdir(os.path.join(parent_dir, item)):
        # Add the full path of the directory to the list
        folder_paths.append(os.path.join(parent_dir, item))

# Print the list of folder paths
print(folder_paths)
print(len(folder_paths))

['/content/index/organization-structure_index', '/content/index/remote-sensing-gis_index', '/content/index/awards_index', '/content/index/messaging_index', '/content/index/nicnet_index', '/content/index/events_index', '/content/index/help_index', '/content/index/domain_registration_index', '/content/index/website-policies_index', '/content/index/command-and-control-centre_index', '/content/index/centralised-aadhaar-vault_index', '/content/index/national-cloud_index', '/content/index/contact-us_index', '/content/index/photo-gallery_index', '/content/index/data-centre_index', '/content/index/security_index', '/content/index/webcast_index', '/content/index/infrastructure_index', '/content/index/video-conferencing_index', '/content/index/Services_index', '/content/index/district-centres_index', '/content/index/Home_index', '/content/index/video-gallery_index', '/content/index/directory_index', '/content/index/profile_index', '/content/index/web-information-manager_index', '/content/index/n

# Combining index

In [ ]:
import os
import json

# List of folders

folders = ['/content/index/organization-structure_index', '/content/index/remote-sensing-gis_index', '/content/index/awards_index', '/content/index/messaging_index', '/content/index/nicnet_index', '/content/index/events_index', '/content/index/help_index', '/content/index/domain_registration_index', '/content/index/website-policies_index', '/content/index/command-and-control-centre_index', '/content/index/centralised-aadhaar-vault_index', '/content/index/national-cloud_index', '/content/index/tabular_data_csv_index', '/content/index/contact-us_index', '/content/index/photo-gallery_index', '/content/index/data-centre_index', '/content/index/security_index', '/content/index/webcast_index', '/content/index/infrastructure_index', '/content/index/video-conferencing_index', '/content/index/Services_index', '/content/index/district-centres_index', '/content/index/Home_index', '/content/index/video-gallery_index', '/content/index/directory_index', '/content/index/profile_index', '/content/index/web-information-manager_index', '/content/index/nkn_index', '/content/index/rti_index', '/content/index/news-update_index', '/content/index/government-local-area-networks-lans_index']
# Initialize merged data
merged_data = {
    "embedding_dict": {},
    "text_id_to_ref_doc_id": {},
    "metadata_dict": {}
}

# Loop through each folder
for folder in folders:
    file_path = os.path.join(folder, 'default__vector_store.json')

    # Load the contents of the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Merge the embedding_dict
    merged_data['embedding_dict'] = {**merged_data['embedding_dict'], **data['embedding_dict']}

    # Merge the text_id_to_ref_doc_id
    merged_data['text_id_to_ref_doc_id'] = {**merged_data['text_id_to_ref_doc_id'], **data['text_id_to_ref_doc_id']}

    # Merge the metadata_dict
    merged_data['metadata_dict'] = {**merged_data['metadata_dict'], **data['metadata_dict']}

# Write the merged data to a new JSON file
with open('/content/combined_index/default__vector_store.json', 'w') as merged_file:
    json.dump(merged_data, merged_file, indent=4)

In [ ]:
import os
import json

# List of folders
folders = ['/content/index/organization-structure_index', '/content/index/remote-sensing-gis_index', '/content/index/awards_index', '/content/index/messaging_index', '/content/index/nicnet_index', '/content/index/events_index', '/content/index/help_index', '/content/index/domain_registration_index', '/content/index/website-policies_index', '/content/index/command-and-control-centre_index', '/content/index/centralised-aadhaar-vault_index', '/content/index/national-cloud_index', '/content/index/tabular_data_csv_index', '/content/index/contact-us_index', '/content/index/photo-gallery_index', '/content/index/data-centre_index', '/content/index/security_index', '/content/index/webcast_index', '/content/index/infrastructure_index', '/content/index/video-conferencing_index', '/content/index/Services_index', '/content/index/district-centres_index', '/content/index/Home_index', '/content/index/video-gallery_index', '/content/index/directory_index', '/content/index/profile_index', '/content/index/web-information-manager_index', '/content/index/nkn_index', '/content/index/rti_index', '/content/index/news-update_index', '/content/index/government-local-area-networks-lans_index']

# Initialize merged data
merged_data = {
    "docstore/metadata": {},
    "docstore/data": {},
    "docstore/ref_doc_info": {}
}

# Loop through each folder
for folder in folders:
    file_path = os.path.join(folder, 'docstore.json')

    # Load the contents of the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Merge the docstore/metadata
    merged_data['docstore/metadata'] = {**merged_data['docstore/metadata'], **data['docstore/metadata']}

    # Merge the docstore/data
    merged_data['docstore/data'] = {**merged_data['docstore/data'], **data['docstore/data']}

    # Merge the docstore/ref_doc_info
    merged_data['docstore/ref_doc_info'] = {**merged_data['docstore/ref_doc_info'], **data['docstore/ref_doc_info']}

# Create the merged JSON data
merged_data_final = {
    "docstore/metadata": merged_data['docstore/metadata'],
    "docstore/data": merged_data['docstore/data'],
    "docstore/ref_doc_info": merged_data['docstore/ref_doc_info']
}

# Write the merged data to a new JSON file
with open('/content/combined_index/docstore.json', 'w') as merged_file:
    json.dump(merged_data_final, merged_file, indent=4)

In [ ]:
import os
import json

# List of folders
folders = ['/content/index/organization-structure_index', '/content/index/remote-sensing-gis_index', '/content/index/awards_index', '/content/index/messaging_index', '/content/index/nicnet_index', '/content/index/events_index', '/content/index/help_index', '/content/index/domain_registration_index', '/content/index/website-policies_index', '/content/index/command-and-control-centre_index', '/content/index/centralised-aadhaar-vault_index', '/content/index/national-cloud_index', '/content/index/tabular_data_csv_index', '/content/index/contact-us_index', '/content/index/photo-gallery_index', '/content/index/data-centre_index', '/content/index/security_index', '/content/index/webcast_index', '/content/index/infrastructure_index', '/content/index/video-conferencing_index', '/content/index/Services_index', '/content/index/district-centres_index', '/content/index/Home_index', '/content/index/video-gallery_index', '/content/index/directory_index', '/content/index/profile_index', '/content/index/web-information-manager_index', '/content/index/nkn_index', '/content/index/rti_index', '/content/index/news-update_index', '/content/index/government-local-area-networks-lans_index']

# Initialize merged data
merged_data = {
    "graph_dict": {}
}

# Loop through each folder
for folder in folders:
    file_path = os.path.join(folder, 'graph_store.json')

    # Load the contents of the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Merge the graph_dict
    merged_data['graph_dict']= {**merged_data['graph_dict'],**data['graph_dict']}

# Write the merged data to a new JSON file
with open('/content/combined_index/graph_store.json', 'w') as merged_file:
    json.dump(merged_data, merged_file, indent=4)

In [ ]:
import os
import json

# List of folders
folders = ['/content/index/organization-structure_index', '/content/index/remote-sensing-gis_index', '/content/index/awards_index', '/content/index/messaging_index', '/content/index/nicnet_index', '/content/index/events_index', '/content/index/help_index', '/content/index/domain_registration_index', '/content/index/website-policies_index', '/content/index/command-and-control-centre_index', '/content/index/centralised-aadhaar-vault_index', '/content/index/national-cloud_index', '/content/index/tabular_data_csv_index', '/content/index/contact-us_index', '/content/index/photo-gallery_index', '/content/index/data-centre_index', '/content/index/security_index', '/content/index/webcast_index', '/content/index/infrastructure_index', '/content/index/video-conferencing_index', '/content/index/Services_index', '/content/index/district-centres_index', '/content/index/Home_index', '/content/index/video-gallery_index', '/content/index/directory_index', '/content/index/profile_index', '/content/index/web-information-manager_index', '/content/index/nkn_index', '/content/index/rti_index', '/content/index/news-update_index', '/content/index/government-local-area-networks-lans_index']

# Initialize merged data
merged_data = {
    "embedding_dict": {},
    "text_id_to_ref_doc_id": {},
    "metadata_dict": {}
}

# Loop through each folder
for folder in folders:
    file_path = os.path.join(folder, 'image__vector_store.json')

    # Load the contents of the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Merge the embedding_dict
    merged_data['embedding_dict'] = {**merged_data['embedding_dict'], **data['embedding_dict']}

    # Merge the text_id_to_ref_doc_id
    merged_data['text_id_to_ref_doc_id'] = {**merged_data['text_id_to_ref_doc_id'], **data['text_id_to_ref_doc_id']}

    # Merge the metadata_dict
    merged_data['metadata_dict'] = {**merged_data['metadata_dict'], **data['metadata_dict']}

# Write the merged data to a new JSON file
with open('/content/combined_index/image_vector_store.json', 'w') as merged_file:
    json.dump(merged_data, merged_file, indent=4)

In [ ]:
import os
import json

# List of folders
folders = ['/content/index/organization-structure_index', '/content/index/remote-sensing-gis_index', '/content/index/awards_index', '/content/index/messaging_index', '/content/index/nicnet_index', '/content/index/events_index', '/content/index/help_index', '/content/index/domain_registration_index', '/content/index/website-policies_index', '/content/index/command-and-control-centre_index', '/content/index/centralised-aadhaar-vault_index', '/content/index/national-cloud_index', '/content/index/tabular_data_csv_index', '/content/index/contact-us_index', '/content/index/photo-gallery_index', '/content/index/data-centre_index', '/content/index/security_index', '/content/index/webcast_index', '/content/index/infrastructure_index', '/content/index/video-conferencing_index', '/content/index/Services_index', '/content/index/district-centres_index', '/content/index/Home_index', '/content/index/video-gallery_index', '/content/index/directory_index', '/content/index/profile_index', '/content/index/web-information-manager_index', '/content/index/nkn_index', '/content/index/rti_index', '/content/index/news-update_index', '/content/index/government-local-area-networks-lans_index']

# Initialize merged data
merged_data = {
    "index_store/data": {
        "vector_index": {
            "__type__": "vector_store",
            "__data__": "{\"index_id\": \"vector_index\", \"summary\": null, \"nodes_dict\": {}, \"doc_id_dict\": {}, \"embeddings_dict\": {}}"
        }
    }
}

# Loop through each folder
for folder in folders:
    file_path = os.path.join(folder, 'index_store.json')

    # Load the contents of the JSON file
    with open(file_path, 'r') as file:
        data = json.load(file)

    # Merge the vector_index data
    vector_index_data = json.loads(merged_data['index_store/data']['vector_index']['__data__'])
    vector_index_data['nodes_dict'] = {**vector_index_data['nodes_dict'], **json.loads(data['index_store/data']['vector_index']['__data__'])['nodes_dict']}
    vector_index_data['doc_id_dict'] = {**vector_index_data['doc_id_dict'], **json.loads(data['index_store/data']['vector_index']['__data__'])['doc_id_dict']}
    vector_index_data['embeddings_dict'] = {**vector_index_data['embeddings_dict'], **json.loads(data['index_store/data']['vector_index']['__data__'])['embeddings_dict']}
    merged_data['index_store/data']['vector_index']['__data__'] = json.dumps(vector_index_data)

# Write the merged data to a new JSON file
with open('/content/combined_index/index_store.json', 'w') as merged_file:
    json.dump(merged_data, merged_file, indent=4)

# Model and Embeddings

In [ ]:
# Initialize settings for LlamaIndex
Settings.llm = Ollama(model="llama3", temperature=0, request_timeout=500.0)

In [ ]:

Settings.embed_model = HuggingFaceEmbedding(model_name="nvidia/NV-Embed-v1", trust_remote_code=True, use_auth_token=True)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/71.2k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


...

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/712k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

# Loading the index from disk

In [ ]:
persist_dir = "/content/combined_index"

storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
index = load_index_from_storage(storage_context)

print(index)

# Chat_engine

In [ ]:
from llama_index.core.storage.chat_store import SimpleChatStore
from llama_index.core.memory import ChatMemoryBuffer

chat_store = SimpleChatStore()
loaded_chat_store = SimpleChatStore.from_persist_path(
    persist_path="chat_store.json"
)

chat_memory = ChatMemoryBuffer.from_defaults(
    token_limit=3000,
    chat_store=chat_store,
    chat_store_key="user1",
)


chat_engine = index.as_chat_engine(
    chat_mode="context",
    system_prompt="You are a helpful AI assistant. You are expert in retrieving the answer for the user input based on the provided context. You are also able to give the source link(url) to each user input.",
    memory=chat_memory, verbose = True
)



In [ ]:
# val_query_response = chat_engine.chat("What can you tell me about the organization structure?")

query_engine = index.as_query_engine()

query = "To whom Ms. D. Lakshmi Prasanna reports to?"
val_query_response = query_engine.query(query)
# print(f"RESPONSE:\n{response}")
# print(val_query_response)

ReadTimeout: timed out

In [ ]:
val_query_response

Response(response='Ms. D. Lakshmi Prasanna reports to Shri M.P. Jayaprasad, Section Officer.', source_nodes=[NodeWithScore(node=TextNode(id_='e645cfc1-9507-47bb-b7c4-bae334fbc203', embedding=None, metadata={'file_path': '/content/tabular_data_csv/Maharashtra State Centre, Mumbai.csv', 'file_name': 'Maharashtra State Centre, Mumbai.csv', 'file_type': 'text/csv', 'file_size': 3488, 'creation_date': '2024-05-28', 'last_modified_date': '2024-05-28'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='ed8eee3c-5067-4e08-a513-4d836a3d9257', node_type=<ObjectType.DOCUMENT: '4'>, metadata={'file_path': '/content/tabular_data_csv/Maharashtra State Centre, Mumbai.csv', 'file_name': 'Maharashtra State Centre, Mumbai.csv'

In [ ]:
# response = chat_engine.chat("what is the published date of Implementation of Aadhar Enabled Biometric Attendance System (AEBAS) in NIC Maharashtra?")

query_engine = index.as_query_engine()

query = "what are the works done by NIC?"
val_query_response = query_engine.query(query)
# print(f"RESPONSE:\n{response}")
# print(response)

In [ ]:
val_query_response

Response(response='According to the provided context information, National Informatics Centre (NIC) has been involved in various works, including:\n\n1. Establishing High Capacity SCPC VSAT Connectivity at Kavarati, Lakshadweep and Port Blair, Andaman & Nicobar Island.\n2. Providing Data Centers Services from National Data Centres at Delhi, Hyderabad, Pune, and Bhubaneswar.\n3. Upgrading the National Data Centre at Delhi with high-speed network backbone, 1.6 Petabyte Enterprise-class storage, high-throughput Network Load Balancers, and Intrusion Prevention Systems.\n4. Implementing Solution for Backup as a Service & Storage as a Service.\n5. Hosting/enhancing ICT infrastructure of national-level projects, including E-office, e-Courts, and e-Transport.\n6. Upgrading Data Centres at Pune and Hyderabad with high-speed network backbone and storage capacity enhanced.\n7. Renovating the National Data Centre at Hyderabad with a capacity of 106 Racks.\n8. Launching National Cloud Services unde

In [ ]:

#print("Jigyasa Response: " + str(val_query_response.response))
node = val_query_response.source_nodes[0]
response_json = {}
response_json['response'] = val_query_response.response
response_json['search_Score'] = node.score

try:
  response_json['url'] = str(node.metadata['URL'])
except  KeyError as e:
  response_json['url'] =  str(file_to_url_mapping[node.metadata['file_name']])







# Queries

In [ ]:
# What are NIC services?
response_json

{'response': 'According to the provided context information, National Informatics Centre (NIC) provides various services including:\n\n1. Electronic Mail (E-Mail) services over NICNET, which is a satellite-based communication network.\n2. SMTP, UUCP, and X.400 email services.\n3. Integration with a X.500 directory for easy searching of e-mail addresses.\n4. Fax messaging through E-Mail.\n5. Messaging services, including core eMail application Gateway services, Short Messaging Service (SMS), Outbound Dialing (OBD), and an IT Platform for citizen engagement (Sampark).\n6. Video Conferencing services for direct interaction with stakeholders.\n7. Webcast services for live/on-demand broadcasts of important events and conferences.\n\nThese services are offered to users across the country, including government departments, ministries, and autonomous bodies.',
 'search_Score': 0.6809562153737795,
 'url': 'https://www.nic.in/servicecontents/messaging/'}

In [ ]:
# What is the published date of Training Program of IVFRT-MMP at FRRO?
response_json

{'response': 'I\'m happy to help! However, I don\'t see any information about a "Training Program of IVFRT-MMP at FRRO" in the provided context. The context only mentions file paths for CSV files and some personal information about Smt. Ireni Akoijam and details about JAWS screen reader. There is no mention of a training program or its published date.',
 'search_Score': 0.47610851359310563,
 'url': 'https://maharashtra.nic.in/rti/'}

In [ ]:
# what is the published date of SAMPARK-NIC eGOV mobile App?
response_json

{'response': 'The published date of SAMPARK – NIC eGov Mobile App is July 31, 2021.',
 'search_Score': 0.683033538785289,
 'url': 'https://maharashtra.nic.in/news-update/'}

In [ ]:
# what is the published date of SIMNIC?
response_json

{'response': 'According to the provided context information, the published date of SIMNIC is August 2, 2021.',
 'search_Score': 0.5385759364469261,
 'url': 'https://maharashtra.nic.in/news-update/'}

In [ ]:
# can you tell me about the latest news?
response_json

{'response': "There is no news update provided in the given context. The context appears to be a list of screen reader information, including the name of the screen reader, its website, and whether it's free or commercial. There is no mention of any news updates.",
 'search_Score': 0.47680748668387585,
 'url': 'https://maharashtra.nic.in/help/'}

In [ ]:
# who is the SIO?
response_json

{'response': 'According to the provided context information, the CISO (Chief Information Security Officer) is responsible for the security of the National Informatics Centre (NIC).',
 'search_Score': 0.6270658935981352,
 'url': 'https://www.nic.in/servicecontents/nkn/'}

In [ ]:
# what are the events?
response_json

{'response': "According to the provided context information, the events that are covered using NIC Webcast services include:\n\n1. Union Budget speech\n2. President's address to the nation\n3. Prime Minister's Mann Ki Baat & other speeches\n4. Independence and Republic Day celebrations at New Delhi\n5. Air Force Day\n6. Dance and cultural Festivals\n7. PIB press conferences\n8. NIC Knowledge sharing\n9. NKN events\n10. Proceedings of state assemblies\n11. National and international events/conferences such as:\n\t* Make in India\n\t* Skill India\n\t* Start-up India\n\t* Digital India\n\t* International Yoga Day",
 'search_Score': 0.4722525002107816,
 'url': 'https://maharashtra.nic.in/infrastructure/'}

In [ ]:
# whom did Ms. D. Lakshmi Prasanna reports to?
response_json

{'response': "Based on the provided context information, I can see that there is no mention of Ms. D. Lakshmi Prasanna's name or designation in the list of scientists and officials. Therefore, it is not possible to determine who her reporting officer would be based on this information alone.",
 'search_Score': 0.7028144708896884,
 'url': 'https://maharashtra.nic.in/directory/'}

In [ ]:
# what is the organization structure?
response_json

{'response': 'The organization structure appears to be a hierarchical structure with various levels of officials and staff. The top-level officials include Scientists-F, who report to higher-level officials such as Scientist-D or Scientist-C. There are also lower-level officials like Section Officers, Senior Secretariat Assistants, Junior Secretariat Assistants, and Staff Car Drivers. Additionally, there are Multitasking Staff and Steno Gr-II personnel.',
 'search_Score': 0.47507247362784927,
 'url': 'https://maharashtra.nic.in/organization-structure/'}

In [ ]:
# who is the reporting officer of Ms. D. Lakshmi Prasanna?
response_json

{'response': 'There is no mention of a photo gallery in the provided context information. The context appears to be related to screen readers and document types, but it does not include any information about a photo gallery. Therefore, I cannot provide an answer to this query based on the given context.',
 'search_Score': 0.5215259570020727,
 'url': 'https://maharashtra.nic.in/help/'}

In [ ]:
# what is there in the video gallery?
response_json

{'response': 'According to the provided context, the Video Gallery contains videos related to NIC Maharashtra – ICT support during COVID-19.',
 'search_Score': 0.5873177463100648,
 'url': 'https://maharashtra.nic.in/video-gallery/'}

In [ ]:
response_json

{'response': 'According to the provided context information, National Informatics Centre (NIC) has been involved in various works, including:\n\n1. Establishing High Capacity SCPC VSAT Connectivity at Kavarati, Lakshadweep and Port Blair, Andaman & Nicobar Island.\n2. Providing Data Centers Services from National Data Centres at Delhi, Hyderabad, Pune, and Bhubaneswar.\n3. Upgrading the National Data Centre at Delhi with high-speed network backbone, 1.6 Petabyte Enterprise-class storage, high-throughput Network Load Balancers, and Intrusion Prevention Systems.\n4. Implementing Solution for Backup as a Service & Storage as a Service.\n5. Hosting/enhancing ICT infrastructure of national-level projects, including E-office, e-Courts, and e-Transport.\n6. Upgrading Data Centres at Pune and Hyderabad with high-speed network backbone and storage capacity enhanced.\n7. Renovating the National Data Centre at Hyderabad with a capacity of 106 Racks.\n8. Launching National Cloud Services under Meg

# Chat_store

In [ ]:
chat_store.persist(persist_path="chat_store.json")
loaded_chat_store = SimpleChatStore.from_persist_path(
    persist_path="chat_store.json"
)

# Visualizing Reranking

In [ ]:
persist_dir = "/content/combined_index"

storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
index = load_index_from_storage(storage_context)

print(index)

In [ ]:
from llama_index.core.retrievers import VectorIndexRetriever
from llama_index.core import QueryBundle
import pandas as pd
from IPython.display import display, HTML
from copy import deepcopy

pd.set_option("display.max_colwidth", None)

# configure retriever
retriever = VectorIndexRetriever(
    index=index,
    similarity_top_k=10)

def get_retrieved_nodes(
    query_str, reranker
):
    query_bundle = QueryBundle(query_str)

    retrieved_nodes = retriever.retrieve(query_bundle)

    if reranker != "None":
      retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)
    else:
        retrieved_nodes

    return retrieved_nodes


def pretty_print(df):
    return display(HTML(df.to_html().replace("\\n", "<br>")))


def visualize_retrieved_nodes(nodes) -> None:
    result_dicts = []
    for node in nodes:
        node = deepcopy(node)
        node.node.metadata = {}
        node_text = node.node.get_text()
        node_text = node_text.replace("\n", " ")

        result_dict = {"Score": node.score, "Text": node_text}
        result_dicts.append(result_dict)

    pretty_print(pd.DataFrame(result_dicts))

# Comparing with reranker and without reranker

In [4]:
# Define all embeddings and rerankers

RERANKERS = {
    "WithoutReranker": "None",
    "cross-encoder": SentenceTransformerRerank(model="cross-encoder/ms-marco-MiniLM-L-2-v2", top_n=5),
    "bge-reranker-large": SentenceTransformerRerank(model="BAAI/bge-reranker-large", top_n=5)
}

NameError: name 'SentenceTransformerRerank' is not defined

In [ ]:
RERANKERS.items()

dict_items([('WithoutReranker', 'None'), ('cross-encoder', SentenceTransformerRerank(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7ca0e5a8f340>, model='cross-encoder/ms-marco-MiniLM-L-2-v2', top_n=5, device='cuda', keep_retrieval_score=False)), ('bge-reranker-large', SentenceTransformerRerank(callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7c9fec1c8be0>, model='BAAI/bge-reranker-large', top_n=5, device='cuda', keep_retrieval_score=False))])

In [ ]:
query_str = "what are the works done by NIC?"

# Loop over rerankers
for rerank_name, reranker in RERANKERS.items():
    print(f"Running Evaluation for Reranker: {rerank_name}")

    query_bundle = QueryBundle(query_str)

    retrieved_nodes = retriever.retrieve(query_bundle)

    if reranker != "None":
      retrieved_nodes = reranker.postprocess_nodes(retrieved_nodes, query_bundle)
    else:
        retrieved_nodes

    print(f"Visualize Retrieved Nodes for Reranker: {rerank_name}")
    visualize_retrieved_nodes(retrieved_nodes)

Running Evaluation for Reranker: WithoutReranker
Visualize Retrieved Nodes for Reranker: WithoutReranker


Running Evaluation for Reranker: cross-encoder
Visualize Retrieved Nodes for Reranker: cross-encoder


Running Evaluation for Reranker: bge-reranker-large
Visualize Retrieved Nodes for Reranker: bge-reranker-large


# Testing Query_engine with Rerank

In [ ]:
persist_dir = "/content/index/combined_index"

storage_context = StorageContext.from_defaults(persist_dir=persist_dir)
index = load_index_from_storage(storage_context)

print(index)

In [8]:
from llama_index.core.postprocessor import SentenceTransformerRerank
from transformers import AutoTokenizer, AutoModel

# old
rerank = SentenceTransformerRerank(model="BAAI/bge-reranker-large", top_n=5)

# new
model = AutoModel.from_pretrained('nvidia/NV-Embed-v1', trust_remote_code=True)

rerank = SentenceTransformerRerank(model=model, top_n=3)


ValueError: Loading nvidia/NV-Embed-v1 requires you to execute the configuration file in that repo on your local machine. Make sure you have read the code there to avoid malicious use, then set the option `trust_remote_code=True` to remove this error.

In [ ]:
from time import time
query_engine = index.as_query_engine(
    similarity_top_k=10, node_postprocessors=[rerank]
)

now = time()
response = query_engine.query("what are the works done by NIC?")
print(f"Elapsed: {round(time() - now, 2)}s")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Elapsed: 13.26s


# Print the response

In [ ]:
node = response.source_nodes[0]
response_json = {}
response_json['response'] = response.response
response_json['Search_Score'] = node.score

try:
  response_json['url'] = str(node.metadata['URL'])
except  KeyError as e:
  response_json['url'] =  str(file_to_url_mapping[node.metadata['file_name']])



In [ ]:
# What are NIC services?
response_json

{'response': 'NIC (National Informatics Centre) offers a wide range of services which includes multi-gigabit nationwide networks, National Data Centres, National Cloud, pan-India VC infrastructure, Command and Control Centre, multi-layered GIS-based platform, Domain Registration, Webcast, Messaging, Remote Sensing & GIS, Government Local Area Networks (LANs), Video Conferencing, Security, Centralised Aadhaar Vault, and more.',
 'Search_Score': 0.9678765,
 'url': 'https://maharashtra.nic.in/services/'}

In [ ]:
# What is the published date of Training Program of IVFRT-MMP at FRRO?
response_json

{'response': 'I\'m happy to help! However, I don\'t see any information about a "Training Program of IVFRT-MMP at FRRO" or its published date in the provided context. The context appears to be a list of individuals with their names, designations, emails, phone numbers, and IP numbers, but it doesn\'t mention anything about a training program. Therefore, I cannot provide an answer to this query based on the given information.',
 'Search_Score': 0.00010795883,
 'url': 'https://maharashtra.nic.in/directory/'}

In [ ]:
# what is the published date of SIMNIC?
response_json

{'response': 'August 2, 2021',
 'Search_Score': 0.96835625,
 'url': 'https://maharashtra.nic.in/news-update/'}

In [ ]:
# what is the published date of SAMPARK?
response_json

{'response': 'The published date of SAMPARK-NIC eGov Mobile App is July 31, 2021.',
 'Search_Score': 0.99882823,
 'url': 'https://maharashtra.nic.in/news-update/'}

In [ ]:
# What is the news update?
response_json

{'response': 'There is no news update. The provided context appears to be a list of screen reader information, including their names, websites, and whether they are free or commercial. There is no indication of any news-related updates.',
 'Search_Score': 7.6261866e-05,
 'url': 'https://maharashtra.nic.in/help/'}

In [ ]:
# Who is the SIO?
response_json

{'response': 'Based on the provided context information, there is no mention of a person named "SIO". However, we do have two files with paths `/content/tabular_data_csv/SIO.csv` and `/content/tabular_data_csv/APPELLATE AUTHORITY.csv`. Since the query asks about the SIO, I would say that the SIO refers to the file path `/content/tabular_data_csv/SIO.csv`, but there is no information provided about who this person might be.',
 'Search_Score': 0.0002616312,
 'url': 'https://maharashtra.nic.in/directory/'}

In [ ]:
# What are the events?
response_json

{'response': 'According to the provided context information, some of the events mentioned are:\n\n1. Cyber Crime and Cyber Laws Webinar on “Cyber Crime and Cyber Laws” at NIC Maharashtra State Centre, Mumbai (September 28th, 2022)\n2. Superannuation of Sh. Dhanjay Kulkarni, DIO Pune and Ms. Sneha Shula, ADIO Nashik in the Month of April 2022\n3. SIO Maharashtra met new Director General, NIC on 01/06/2022\n4. Hon’ble Minister of Electronics and Information Technology and Railways visit to Mumbai (18.02.2022)\n5. Swachhatta Pakhwada from 01st to 15th February 2022\n\nThese events are listed under the "Events" section on the website.',
 'Search_Score': 0.1354689,
 'url': 'https://maharashtra.nic.in/events/'}

In [ ]:
# what is the organization structure?
response_json

{'response': 'The organization structure consists of various officials with their respective designations, reporting to other officials. The officials include:\n\n1. Ms. D. Lakshmi Prasanna - Scientist-F\nReporting To: Shri Anand Swarup Srivastava\n2. Shri C.Prasanna - Scientist-E\nReporting To: Ms. D. Lakshmi Prasanna\n3. Shri Syed Ajaz Gulab - Scientist-E\nReporting To: Ms. D. Lakshmi Prasanna\n4. Shri Rajendra F. Hatwar - Scientist-D\nReporting To: Ms. D. Lakshmi Prasanna\n5. Shri Kunal Vijay Bhamare - Scientist-D\nReporting To: Shri Rajendra F. Hatwar\n6. Shri Saravana Kumar - Scientist-B\nReporting To: Shri Kunal Vijay Bhamare\n\nAnd so on, with multiple officials listed under various categories such as Scientist-F, Scientist-E, Scientist-D, and more.',
 'Search_Score': 0.7342759,
 'url': 'https://maharashtra.nic.in/organization-structure/'}

In [ ]:
# whom did Ms. D. Lakshmi Prasanna reports to?
response_json

{'response': 'Shri Sunil Digambarro Rao Potekar.',
 'Search_Score': 0.02319305,
 'url': 'https://maharashtra.nic.in/directory/'}

In [ ]:
# what is there in the video gallery?
response_json

{'response': "According to the provided context information, the Video Gallery contains a collection of videos showcasing NIC Maharashtra's ICT support during COVID-19.",
 'Search_Score': 0.024777586,
 'url': 'https://maharashtra.nic.in/video-gallery/'}

In [ ]:
response_json

{'response': "Based on the provided context information, National Informatics Centre (NIC) provides various services and platforms to facilitate e-Governance applications in Government Ministries/Departments at the Centre, States, Districts, and Block level. Some of the notable works done by NIC include:\n\n1. Providing different kinds of E-mail services over NICNET, which is NIC's satellite-based communication network.\n2. Establishing a robust Messaging framework that includes core eMail application Gateway services, Short Messaging Service (SMS), OBD (Outbound Dialing), and an IT Platform for citizen engagement (Sampark).\n3. Developing and hosting various websites and portals, such as the NIC website, which provides information on NIC's services, products, and platforms.\n4. Providing institutional linkages across all the Ministries /Departments of the Central Government, State Governments, Union Territories, and District administrations of the country through its Information and C